# Elliptic Curve Cryptography (ECC)

**What is ECC?**

- ECC is a form of public-key cryptography based on the mathematics of elliptic curves over finite fields.
- It achieves equivalent security to RSA with much smaller key sizes, making it faster and more resource-efficient.

---

**How It Works (Intuition)**

- **Key Pair Generation:**
  - **Private Key:** A randomly chosen number.
  - **Public Key:** A point on the curve obtained by multiplying a predefined base point (**G**) by the private key.

- **Security Basis:**  
  The hardness of the Elliptic Curve **Discrete Logarithm** Problem (ECDLP):
  - Given a base point **P** and a point **Q = k * P**, it is computationally infeasible to recover **k**.

- **Applications:**
  - Key exchange (ECDH)
  - Digital signatures (ECDSA, EdDSA)

---

**Communication Flow (ECDH Example)**

```text
Alice                         Bob                        Eve
---------------------------------------------------------------
Private key: a                Private key: b
Public key: A = a * G         Public key: B = b * G
      |---- A --------------->|---- B ---------------> (observes A, B, G)
Shared secret: a * B          Shared secret: b * A
             -> (ab) * G       -> (ab) * G
```

- **Shared Secret:** Both Alice and Bob compute the same shared point **(ab) * G**.
- **Eve's Limitation:** Though Eve can see **A**, **B**, and **G**, she cannot derive **(ab) * G** without solving the ECDLP.

---

**ECC Parameters**

- **Curve Equation:**  
  `y² = x³ + ax + b (mod p)`  
  where **a**, **b**, and **p** are curve parameters, with **p** being a prime.

- **G:** The base point (generator) on the curve.

- **n:** The order of the base point.

- **Private Key:**  
  A random integer `d` in the range `[1, n-1]`.

- **Public Key:**  
  `Q = d * G` (obtained via scalar multiplication).

---

**Where It’s Used**

- **TLS/HTTPS:**  
  Utilizes curves like **X25519/ECDHE** for secure key exchange.
  
- **Messaging Apps:**  
  Signal employs **Curve25519**.
  
- **Cryptocurrencies:**  
  Bitcoin and Ethereum use **secp256k1** for digital signatures.
  
- **Modern Authentication:**  
  Commonly used in SSH, JWT, and various secure authentication protocols.

---

**Security Considerations**

- **Use Standard Curves:**  
  - **Curve25519 / X25519:** For key exchange.
  - **Ed25519:** For digital signatures.
  - **secp256r1 (NIST P-256):** Widely adopted.
  - **secp256k1:** Used in Bitcoin and other cryptocurrencies.
  
- **Avoid Custom Parameters:**  
  Always use well-reviewed, standard curve parameters rather than homemade ones.

- **Enhance Key Exchange:**  
  Combine ECDH with a Key Derivation Function (KDF) and authenticated encryption to ensure confidentiality and integrity.

# Demo (Toy, educational; not secure)

This shows scalar multiplication on a small elliptic curve.
(Real curves use very large primes; this is just to see the math.)

In [3]:
# Tiny elliptic curve over a prime field
# Equation: y^2 = x^3 + ax + b mod p
p = 97
a = 2
b = 3

# Base point (generator)
G = (3, 6)

# --- Elliptic curve point operations ---
def inv_mod(k, p):
    # modular inverse
    return pow(k, -1, p)

def point_add(P, Q):
    if P is None: return Q
    if Q is None: return P
    (x1, y1), (x2, y2) = P, Q
    if x1 == x2 and y1 != y2:
        return None
    if P == Q:
        m = (3*x1*x1 + a) * inv_mod(2*y1, p) % p
    else:
        m = (y2 - y1) * inv_mod(x2 - x1, p) % p
    x3 = (m*m - x1 - x2) % p
    y3 = (m*(x1 - x3) - y1) % p
    return (x3, y3)

def scalar_mult(k, P):
    R = None
    for bit in bin(k)[2:]:
        R = point_add(R, R)
        if bit == "1":
            R = point_add(R, P)
    return R

# --- Alice and Bob key exchange ---
import secrets
n = 5  # small order for demo (toy only!)
a_priv = secrets.randbelow(n) + 1
b_priv = secrets.randbelow(n) + 1
A = scalar_mult(a_priv, G)
B = scalar_mult(b_priv, G)

# Shared secret
S_alice = scalar_mult(a_priv, B)
S_bob   = scalar_mult(b_priv, A)

print("Alice secret:", S_alice)
print("Bob secret:  ", S_bob)


Alice secret: (3, 6)
Bob secret:   (3, 6)


# Demo — Real ECDH (with cryptography)

This is how to do ECC securely in practice.

In [6]:
# pip install cryptography
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend

# Generate Alice & Bob's keys on NIST P-256
alice_priv = ec.generate_private_key(ec.SECP256R1())
bob_priv   = ec.generate_private_key(ec.SECP256R1())

# Public keys
alice_pub = alice_priv.public_key()
bob_pub   = bob_priv.public_key()

# Shared secrets (must match)
alice_shared = alice_priv.exchange(ec.ECDH(), bob_pub)
bob_shared   = bob_priv.exchange(ec.ECDH(), alice_pub)
assert alice_shared == bob_shared

# Derive symmetric key from shared secret
hkdf = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b"ecdh-demo",
    backend=default_backend()
)
key = hkdf.derive(alice_shared)

print("Shared secret established.")
print("Derived key (32 bytes, hex):", key.hex())


Shared secret established.
Derived key (32 bytes, hex): 7d6bff15c6182bf261f77307e5e4439447ca2b3f9c2f8ab853ffa7e30fa5fb86
